In [1]:
import os
import networkx as nx
import numpy as np

In [2]:
g = nx.read_graphml('data/NET-btc-heur_0-week/2010-09-06.graphml')

In [14]:

def bow_tie(g):
    
    #set states for different groups
    disconnected = 0
    in_node = 1
    out_node = 2
    tendril = 3
    tube = 4
    fringe = 5
    ssc = 6

    # set state of all nodes to None
    for i in g.nodes():
        g.nodes[i]["state"] = None

    # calculate ssc
    strongly_connected = max(nx.strongly_connected_components(g), key=len)

    for node in strongly_connected:
        g.nodes[node]["state"] = ssc
        # get all out nodes by looking at succesors from ssc
        for out in g.successors(node):
            if g.nodes[out]["state"] == None:
                g.nodes[out]["state"] = out_node
        
        # get all in nodes by looking at predecessors from ssc
        for into in g.predecessors(node):
            if g.nodes[into]["state"] == None:
                g.nodes[into]["state"] = in_node
    
    # after calculating in, out and ssc next step was get tube, fringe and tendril
    
    for node in g.nodes():
        if g.nodes[node]["state"] == in_node:
            # go through successors and do disctinction of ssc, tube and tendril
            for tend_from_in in g.successors(node):
                if g.nodes[tend_from_in]["state"] != ssc:
                    for tube_opt in g.successors(tend_from_in):
                        # check if tendril is connected to out to get all tube and assign them
                        if g.nodes[tube_opt]["state"] == out_node:
                            g.nodes[tube_opt]["state"] = tube
                    # loop through tendril to get first half of all fringe
                    g.nodes[tend_from_in]["state"] = tendril
                    for fringe_in in g.predecessors(tend_from_in):
                        if g.nodes[fringe_in]["state"] != in_node:
                            g.nodes[fringe_in]["state"] = fringe

        if g.nodes[node]["state"] == out_node:
            # go through predecessors and do disctionction of ssc, tube and tendril
            for tend_to_out in g.predecessors(node):
                # sort out all ssc and tubes and assign to tendril(_to_out)
                if g.nodes[tend_to_out]["state"] != ssc:
                    if g.nodes[tend_to_out]["state"] != tube:
                        g.nodes[tend_to_out]["state"] = tendril
                        # loop through tend to outs to get the second part of all fringe
                        for fringe_out in g.successors(tend_to_out):
                            if g.nodes[fringe_out]["state"] != out_node:
                                g.nodes[fringe_out]["state"] = fringe

        # assign all not assigned nodes as disconnected
        if g.nodes[node]["state"] == None:
            g.nodes[node]["state"] = disconnected
            
    in_list = np.where([g.nodes[i]['state'] == in_node for i in g.nodes()])[0]
    out_list = np.where([g.nodes[i]['state'] == out_node for i in g.nodes()])[0]
    ssc_list = np.where([g.nodes[i]['state'] == ssc for i in g.nodes()])[0]
    disc_list = np.where([g.nodes[i]['state'] == disconnected for i in g.nodes()])[0]
    tube_list = np.where([g.nodes[i]['state'] == tube for i in g.nodes()])[0]
    tendril_list = np.where([g.nodes[i]['state'] == tendril for i in g.nodes()])[0]
    fringe_list = np.where([g.nodes[i]['state'] == fringe for i in g.nodes()])[0]
    
    return (round(len(in_list)/len(g),4),
            round(len(out_list)/len(g),4),
            round(len(ssc_list)/len(g),4),
            round(len(disc_list)/len(g),4), 
            round(len(tendril_list)/len(g),4), 
            round(len(tube_list)/len(g),4), 
            round(len(fringe_list)/len(g),4))

In [11]:
test = bow_tie(g)

In [12]:
test

(0.0013, 0.0638, 0.0721, 0.8452, 0.0114, 0.0, 0.0062)

In [13]:
sum(test)

0.9999999999999999